In [408]:
import numpy as np
from scipy.optimize import minimize


In [409]:
def func(x):
    if x.ndim == 2:
        r = np.linalg.norm(x, axis = 1)

    elif x.ndim == 1:
        r = np.linalg.norm(x)
    
    return r**2 - 2*r + 7

def Nelder_Mead(x_init, step_size = 2):
    
    n = len(x_init)

    if n >= 2:

        alpha = 1
        beta  = 1 + 2/n
        gamma = 0.75 - 1/(2*n)
        delta = 1 - 1/n
    
    else:

        alpha = 1
        beta  = 2
        gamma = 0.5
        delta = 0.5


    x_simplex = np.zeros((n + 1, n))

    x_simplex[0] = x_init

    for i in range(n):
        
        ei = np.zeros((n,))
        ei[i] = 1

        x_simplex[i + 1] = x_simplex[0] + step_size*ei
    
    for iter in range(100):

        f = func(x_simplex)

        f_with_index = np.column_stack([f, np.arange(n+1)])

        f_with_index = f_with_index[np.argsort(f_with_index[:,0])]

        f = f_with_index[:,0]

        f_index = f_with_index[:,1].astype(int)

        x_centroid = np.mean(x_simplex, axis = 0)

        xr = x_centroid + alpha*(x_centroid - x_simplex[f_index[-1]])

        fr = func(xr)

        if f[0] <= fr < f[n-1]:

            x_simplex[f_index[-1]] = xr

        elif fr < f[0]:

            xe = x_centroid + beta*(xr - x_centroid)

            fe = func(xe)

            if fe < fr:

                x_simplex[f_index[-1]] = xe
            
            else:

                x_simplex[f_index[-1]] = xr
            
        elif f[n-1] <= fr < f[n]:

            xoc = x_centroid + gamma*(xr - x_centroid)

            foc = func(xoc)

            if foc <= fr:

                x_simplex[f_index[-1]] = xoc
            
            else:

                x_simplex[1:] = x_simplex[0] + delta*(x_simplex[1:] - x_simplex[0])
            
        elif fr >= f[n]:

            xic = x_centroid - gamma*(xr - x_centroid)

            fic = func(xic)

            if fic <= fr:

                x_simplex[f_index[-1]] = xic
            
            else:

                x_simplex[1:] = x_simplex[0] + delta*(x_simplex[1:] - x_simplex[0])

    print(x_simplex, f)


In [418]:
Nelder_Mead(np.zeros((1,)))

options ={
    'initial_simplex': np.array([[1], [7]])
}

minimize(method='Nelder-Mead', fun = func, x0 = np.zeros((1,)), options=options)

[[1.125]
 [1.125]] [6.015625 6.015625]


       message: Optimization terminated successfully.
       success: True
        status: 0
           fun: 6.0
             x: [ 1.000e+00]
           nit: 17
          nfev: 35
 final_simplex: (array([[ 1.000e+00],
                       [ 9.999e-01]]), array([ 6.000e+00,  6.000e+00]))